# LABORATORIO #2 - Deteccion de SPAM
 Autor: Javier Estuardo 19202
 
 Autor: Marco Ramirez   19588

### Ingenieria de caracteristicas


* #### Exploracion de datos
    Para el laboratorio se proporcionan dos datasets distintos. Revise la data y realice las operaciones
necesarias para unificar los datasets y que el dataset final contenga el mensaje del correo y la etiqueta
que indique si es SPAM o no.
Muestre ejemplos de los datasets individuales y del dataset final


In [99]:
# Carga del primer dataset

import pandas as pd

df1 = pd.read_csv('completeSpamAssassin.csv')
df1.sample(n=5).head() # print a random sample of the DataFrame



,Unnamed: 0,Body,Label
4872,4872,Could you please help me how to set up a dial-...,0
2850,2850,You can also duplicate thiswithMsgChange - nos...,0
3619,3619,"[Guido]\n> There seem to be two ""drivers"" for ...",0
2241,2241,">>>>> ""O"" == Owen Byrne writes: O> From th...",0
3834,3834,URL: http://www.mozillazine.org/weblogs/hyatt/...,0


In [100]:
# Carga del segundo dataset

df2 = pd.read_csv('enronSpamSubset.csv')
df2.sample(n=5).head() # print a random sample of the DataFrame


,Unnamed: 0.1,Unnamed: 0,Body,Label
5467,30740,30740,"Subject: hpl nom for may 18 , 2001\n ( see att...",0
288,5823,5823,Subject: do you love suffering for nothing\n i...,1
1363,6586,6586,Subject: network cable tester and free ! rj - ...,1
4417,10912,10912,Subject: popular soft\n popular cheap soft bel...,1
5333,18342,18342,Subject: ny march real options conference\n pl...,0


In [101]:
# Carga del dataset final

df1.drop(['Unnamed: 0'], axis=1, inplace=True)
df2.drop(['Unnamed: 0'], axis=1, inplace=True)
df2.drop(['Unnamed: 0.1'], axis=1, inplace=True)

df = pd.concat([df1, df2], ignore_index=True)




In [102]:
df.head() # print the first 5 rows of the DataFrame

#Save as CSV

df.to_csv('dataSetFinal.csv', index=False)

db = pd.read_csv('dataSetFinal.csv')
db.head()



,Body,Label
0,\nSave up to 70% on Life Insurance.\nWhy Spend...,1
1,1) Fight The Risk of Cancer!\nhttp://www.adcli...,1
2,1) Fight The Risk of Cancer!\nhttp://www.adcli...,1
3,##############################################...,1
4,I thought you might like these:\n1) Slim Down ...,1


* #### Preprocesamiento

    Aplique las técnicas de pre – procesamiento de lenguaje natural que considere necesarias (conversión
de minúsculas, mayúsculas, eliminación de acentos, expansión de contracciones, eliminación de stop
words, etc.)

In [103]:
# First we need to make lowercase all the words in the dataset

db['Body'] = db['Body'].astype(str).str.lower()
db.head()


,Body,Label
0,\nsave up to 70% on life insurance.\nwhy spend...,1
1,1) fight the risk of cancer!\nhttp://www.adcli...,1
2,1) fight the risk of cancer!\nhttp://www.adcli...,1
3,##############################################...,1
4,i thought you might like these:\n1) slim down ...,1


In [104]:
#Se remueve todos los caracteres especiales, links numeros y simbolos

import re

db['Body']= db['Body'].apply(lambda x: re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", " ", x))
db['Body']= db['Body'].apply(lambda x: re.sub("^\d+\s|\s\d+\s|\s\d+$", " ", x))




In [105]:
#Tokens

from nltk.tokenize import RegexpTokenizer

regexp = RegexpTokenizer('\w+')

db['text_token']=db['Body'].apply(regexp.tokenize)
db.head(3)

,Body,Label,text_token
0,save up to on life insurance why spend more...,1,"[save, up, to, on, life, insurance, why, spend..."
1,fight the risk of cancer slim down guar...,1,"[fight, the, risk, of, cancer, slim, down, gua..."
2,fight the risk of cancer slim down guar...,1,"[fight, the, risk, of, cancer, slim, down, gua..."


In [106]:
#Stopwords
import nltk

nltk.download('stopwords')
from nltk.corpus import stopwords

# Make a list of english stopwords
stopwords = nltk.corpus.stopwords.words("english")

# Extend the list with your own custom stopwords
my_stopwords = ['https']
stopwords.extend(my_stopwords)

# Remove stopwords
db['text_token'] = db['text_token'].apply(lambda x: [item for item in x if item not in stopwords])
db.head(3)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\maqui\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Body,Label,text_token
0,save up to on life insurance why spend more...,1,"[save, life, insurance, spend, life, quote, sa..."
1,fight the risk of cancer slim down guar...,1,"[fight, risk, cancer, slim, guaranteed, lose, ..."
2,fight the risk of cancer slim down guar...,1,"[fight, risk, cancer, slim, guaranteed, lose, ..."


In [107]:
#Remove infrequent words
db['text_string'] = db['text_token'].apply(lambda x: ' '.join([item for item in x if len(item)>2]))
db[['Body', 'text_token', 'text_string']].head()

,Body,text_token,text_string
0,save up to on life insurance why spend more...,"[save, life, insurance, spend, life, quote, sa...",save life insurance spend life quote savings e...
1,fight the risk of cancer slim down guar...,"[fight, risk, cancer, slim, guaranteed, lose, ...",fight risk cancer slim guaranteed lose lbs day...
2,fight the risk of cancer slim down guar...,"[fight, risk, cancer, slim, guaranteed, lose, ...",fight risk cancer slim guaranteed lose lbs day...
3,...,"[adult, club, offers, free, membership, instan...",adult club offers free membership instant acce...
4,i thought you might like these slim down g...,"[thought, might, like, slim, guaranteed, lose,...",thought might like slim guaranteed lose lbs da...


In [108]:
#Create a list of all words
all_words = ' '.join([word for word in db['text_string']])
#Tokenize 
tokenized_words = nltk.tokenize.word_tokenize(all_words)
from nltk.probability import FreqDist

fdist = FreqDist(tokenized_words)
#Palabras mas comunes encontradas
fdist.most_common(10)


[('enron', 19276),
 ('subject', 15367),
 ('ect', 11008),
 ('com', 10698),
 ('company', 8271),
 ('please', 7973),
 ('new', 7530),
 ('one', 7424),
 ('would', 7296),
 ('get', 6851)]

In [94]:
#Palabras menos comunes encontradas
fdist.most_common()[-10:]

[('dupuy', 1),
 ('postlethwaite', 1),
 ('marryott', 1),
 ('madhup', 1),
 ('skellett', 1),
 ('umanoff', 1),
 ('emeeting', 1),
 ('qualcom', 1),
 ('bloombergs', 1),
 ('securityrequests', 1)]

In [109]:
#En base a lo anterior se considero eliminar las palabras que se repiten de manera frecuente ya que estas siempre estaran en el correo.

db['text_string']=  db['text_string'].str.replace("subject", " ")
db['text_string']=  db['text_string'].str.replace("enron", " ")
db['text_string']=  db['text_string'].str.replace("com", " ")
db['text_string']=  db['text_string'].str.replace("email", " ")
#Create a list of all words
all_words = ' '.join([word for word in db['text_string']])
#Tokenize 
tokenized_words = nltk.tokenize.word_tokenize(all_words)
from nltk.probability import FreqDist

fdist = FreqDist(tokenized_words)
#Palabras mas comunes encontradas
fdist.most_common(10)

[('ect', 11008),
 ('pany', 8310),
 ('please', 7974),
 ('new', 7559),
 ('one', 7424),
 ('would', 7296),
 ('get', 6851),
 ('time', 6478),
 ('information', 6371),
 ('list', 5890)]

In [110]:
db['text_string_fdist'] = db['text_token'].apply(lambda x: ' '.join([item for item in x if fdist[item] >= 1 ]))
db[['Body', 'text_token', 'text_string', 'text_string_fdist']].head()


,Body,text_token,text_string,text_string_fdist
0,save up to on life insurance why spend more...,"[save, life, insurance, spend, life, quote, sa...",save life insurance spend life quote savings e...,save life insurance spend life quote savings e...
1,fight the risk of cancer slim down guar...,"[fight, risk, cancer, slim, guaranteed, lose, ...",fight risk cancer slim guaranteed lose lbs day...,fight risk cancer slim guaranteed lose lbs day...
2,fight the risk of cancer slim down guar...,"[fight, risk, cancer, slim, guaranteed, lose, ...",fight risk cancer slim guaranteed lose lbs day...,fight risk cancer slim guaranteed lose lbs day...
3,...,"[adult, club, offers, free, membership, instan...",adult club offers free membership instant acce...,adult club offers free membership instant acce...
4,i thought you might like these slim down g...,"[thought, might, like, slim, guaranteed, lose,...",thought might like slim guaranteed lose lbs da...,thought might like slim guaranteed lose lbs da...


In [111]:
#Lematizacion
nltk.download('wordnet')
nltk.download('omw-1.4')

from nltk.stem import WordNetLemmatizer

wordnet_lem = WordNetLemmatizer()

db['text_string_lem'] = db['text_string_fdist'].apply(wordnet_lem.lemmatize)
db

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\maqui\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\maqui\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


,Body,Label,text_token,text_string,text_string_fdist,text_string_lem
0,save up to on life insurance why spend more...,1,"[save, life, insurance, spend, life, quote, sa...",save life insurance spend life quote savings e...,save life insurance spend life quote savings e...,save life insurance spend life quote savings e...
1,fight the risk of cancer slim down guar...,1,"[fight, risk, cancer, slim, guaranteed, lose, ...",fight risk cancer slim guaranteed lose lbs day...,fight risk cancer slim guaranteed lose lbs day...,fight risk cancer slim guaranteed lose lbs day...
2,fight the risk of cancer slim down guar...,1,"[fight, risk, cancer, slim, guaranteed, lose, ...",fight risk cancer slim guaranteed lose lbs day...,fight risk cancer slim guaranteed lose lbs day...,fight risk cancer slim guaranteed lose lbs day...
3,...,1,"[adult, club, offers, free, membership, instan...",adult club offers free membership instant acce...,adult club offers free membership instant acce...,adult club offers free membership instant acce...
4,i thought you might like these slim down g...,1,"[thought, might, like, slim, guaranteed, lose,...",thought might like slim guaranteed lose lbs da...,thought might like slim guaranteed lose lbs da...,thought might like slim guaranteed lose lbs da...
...,...,...,...,...,...,...
16041,subject monday nd oct louise do you have ...,0,"[subject, monday, nd, oct, louise, half, hour,...",monday oct louise half hour free diary see w...,monday oct louise half hour free diary see whi...,monday oct louise half hour free diary see whi...
16042,subject missing bloomberg deals stephanie ...,0,"[subject, missing, bloomberg, deals, stephanie...",missing bloomberg deals stephanie believe fo...,missing bloomberg deals stephanie believe foun...,missing bloomberg deals stephanie believe foun...
16043,subject eops salary survey questionnaire we ...,0,"[subject, eops, salary, survey, questionnaire,...",eops salary survey questionnaire need establ...,eops salary survey questionnaire need establis...,eops salary survey questionnaire need establis...
16044,subject q comparison hi louise i have a c...,0,"[subject, q, comparison, hi, louise, compariso...",parison louise parison first two weeks sti...,hi louise first two weeks still early course h...,hi louise first two weeks still early course h...


In [112]:
#Ahora ya tenemos el dataset limpio y listo para ser utilizado en el modelo de clasificacion
db.drop(['Body'], axis=1, inplace=True)
db.drop(['text_token'], axis=1, inplace=True)
db.drop(['text_string'], axis=1, inplace=True)
db.drop(['text_string_fdist'], axis=1, inplace=True)



db


,Label,text_string_lem
0,1,save life insurance spend life quote savings e...
1,1,fight risk cancer slim guaranteed lose lbs day...
2,1,fight risk cancer slim guaranteed lose lbs day...
3,1,adult club offers free membership instant acce...
4,1,thought might like slim guaranteed lose lbs da...
...,...,...
16041,0,monday oct louise half hour free diary see whi...
16042,0,missing bloomberg deals stephanie believe foun...
16043,0,eops salary survey questionnaire need establis...
16044,0,hi louise first two weeks still early course h...


In [123]:
# Guardamos el dataset limpio
db.rename(columns={'text_string_lem': 'Body'}, inplace=True)
db = db[['Body', 'Label']]

df = db.to_csv('dataSetFinalClean.csv', index=False)


In [124]:
df = pd.read_csv('dataSetFinalClean.csv')
df.head()

,Body,Label
0,save life insurance spend life quote savings e...,1
1,fight risk cancer slim guaranteed lose lbs day...,1
2,fight risk cancer slim guaranteed lose lbs day...,1
3,adult club offers free membership instant acce...,1
4,thought might like slim guaranteed lose lbs da...,1


* #### Representacion de texto
    Utilice los modelos de BoG (para n = 1,2) y TF-IDF. Muestre algunos ejemplos de los mensajes en su
representación numérica.

In [125]:
# BoG (Bag of n-grams) with n=1 and 2
import pandas as pd
import numpy as np
import re
from nltk.util import ngrams
from sklearn.feature_extraction.text import CountVectorizer
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')



stop_words = nltk.corpus.stopwords.words('english')

def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    # tokenize document
    tokens = nltk.word_tokenize(doc)
    # filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

normalize_corpus = np.vectorize(normalize_document)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\maqui\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\maqui\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [157]:

#Count amount of 0 or 1


norm_corpus = normalize_corpus(db['Body'][0:1000])
norm_corpus



array(['save life insurance spend life quote savings ensuring family financial security important life quote savings makes buying life insurance simple affordable provide free access best lowest rates life quote savings fast easy saves money let us help get started best values country new coverage save hundreds even thousands dollars requesting free quote lifequote savings service take less minutes shop save types life insurance click free quote protecting family best investment ever make receipt error wish removed list please click type remove reside state prohibits e mail solicitations insurance please disregard',
       'fight risk cancer slim guaranteed lose lbs days get child support deserve free legal advice join web fastest growing singles start private photo album online wonderful day offer manager prizemamaif wish leave list please use link irish linux users group ilug linux un subscription information list maintainer listmaster linux',
       'fight risk cancer slim guarantee

In [162]:

bv = CountVectorizer(ngram_range=(1,2))
bv_matrix = bv.fit_transform(norm_corpus)


In [163]:

bv_matrix = bv_matrix.toarray()
vocabulario = bv.get_feature_names_out()
bog=pd.DataFrame(bv_matrix, columns=vocabulario)
bog

,001zrgsjtxqsod7antwgmpny5hd15v0q7rboxnpow3xmnc6v,001zrgsjtxqsod7antwgmpny5hd15v0q7rboxnpow3xmnc6v pidjkxpky,0239ur,0239ur tbf1rpua9qesejltwxk1sk8ekr7fcaogmsnrrmc5bwttytbvz,0239usvfk2e7z5lhune0z1wjydfx02x,0239usvfk2e7z5lhune0z1wjydfx02x z5nryeiwioxwejpzvndihjaugzuzxv,02r9uiu3yby7kxx3d2ietv,02r9uiu3yby7kxx3d2ietv tpwjfkeoy9hpj0wuummmscdyyyu63og8fx,031syuhvsv1ml2i2xx0fxhjnbtxxxkumnx,031syuhvsv1ml2i2xx0fxhjnbtxxxkumnx hlxqtsswfwqssss5i5pox7elxwnodu3m2q3sunng1knpgqo0us8m4li0twknmpny44ovq1m1fn7y,...,zzvrulbbwiprjx5,zzvrulbbwiprjx5 jnxiqsr7n6zy45y,zzzz,zzzz jmason,zzzz mortgage,zzzz neilgarner,zzzz spamassassin,zzzz want,zzzzason,zzzzason org
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


TF-IDF


In [164]:
from sklearn.feature_extraction.text import TfidfVectorizer

tv = TfidfVectorizer(min_df=0., max_df=1., use_idf=True)
tv_matrix = tv.fit_transform(norm_corpus)
tv_matrix = tv_matrix.toarray()

vocabulario = tv.get_feature_names_out()
tf=pd.DataFrame(np.round(tv_matrix, 2), columns=vocabulario)
tf

,001zrgsjtxqsod7antwgmpny5hd15v0q7rboxnpow3xmnc6v,0239ur,0239usvfk2e7z5lhune0z1wjydfx02x,02r9uiu3yby7kxx3d2ietv,031syuhvsv1ml2i2xx0fxhjnbtxxxkumnx,03xui5yi5loebl1ast4plfmoottvlewbw8ppluvllunekrhjfltr9ivl6dkyqfjxamsk0mk2kwnh,06cjzuupxmhp0shox12cytzx7zvxtplu9i4rwytel1t5govmzdtxplp4s,06r9ujbqirqvlgdykjzszt21yqmmgvndgsnh7fe1nkztpa8vym09qs,06u3slnj7zqboue,07nurg5dfenhlkc7gozvxsp,...,zynt,zynzi9jlfbztqinca5c,zyrzvnmg,zz0imci,zzh45lk2lum4fdfvbvkemz8cwigaaaaaaaabyulclk,zzjbgsasir5ebhipcln3midcbpswccmr7ctvlw75nku00lxyri7knjwz4lwt9wl9p9y2uunaagsj,zzr5xu2rtijnpzcej6jarj10vkyml5n4jdxq,zzvrulbbwiprjx5,zzzz,zzzzason
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.05,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0


In [165]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_matrix = cosine_similarity(tv_matrix)
similarity_df = pd.DataFrame(similarity_matrix)
similarity_df

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,1.000000,0.025452,0.027325,0.079757,0.026723,0.066701,0.025436,0.231758,0.075086,0.034995,...,0.042445,0.094027,0.212160,0.027213,0.008532,0.0,0.039339,0.421227,0.047729,0.011946
1,0.025452,1.000000,0.838802,0.026182,0.707885,0.076006,0.041588,0.001720,0.072380,0.016373,...,0.073407,0.018495,0.046777,0.046268,0.000000,0.0,0.016313,0.043798,0.000000,0.013249
2,0.027325,0.838802,1.000000,0.031414,0.496468,0.079233,0.049900,0.002064,0.073854,0.012147,...,0.048586,0.015569,0.041095,0.050658,0.000000,0.0,0.015007,0.052551,0.000000,0.015897
3,0.079757,0.026182,0.031414,1.000000,0.024387,0.058389,0.013455,0.047272,0.069935,0.019818,...,0.276390,0.039277,0.050308,0.018490,0.024443,0.0,0.024691,0.072069,0.015203,0.019440
4,0.026723,0.707885,0.496468,0.024387,1.000000,0.045110,0.020790,0.001806,0.046299,0.017191,...,0.072550,0.019419,0.037695,0.048580,0.000000,0.0,0.007991,0.036189,0.000000,0.013911
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000
996,0.039339,0.016313,0.015007,0.024691,0.007991,0.151971,0.014670,0.002801,0.163930,0.209983,...,0.078245,0.044281,0.083980,0.061841,0.054912,0.0,1.000000,0.006196,0.013786,0.015799
997,0.421227,0.043798,0.052551,0.072069,0.036189,0.043646,0.021682,0.271267,0.056135,0.000000,...,0.047875,0.054678,0.194444,0.038468,0.022234,0.0,0.006196,1.000000,0.041699,0.011027
998,0.047729,0.000000,0.000000,0.015203,0.000000,0.028300,0.037675,0.000000,0.024738,0.022581,...,0.013837,0.031419,0.038225,0.028922,0.000000,0.0,0.013786,0.041699,1.000000,0.013850


### Implementacion del modelo


* Separacion de datos : 

    * Datos de entrenamiento: 70% 
    * Datos de prueba: 30%




* Implementacion del modelo naive bayes

* Resultados del modelo

#### Conclusion
Compare los valores para cada modelo de representación numérico. ¿Qué modelo produjo el mejor
resultado, BoG o TF-IDF? ¿A que se debe la mejora? Explique los valores obtenidos en las métricas
para el mejor modelo.